# 네이버 지식인 크롤링 - 여친선물

In [1]:
!apt-get update > /dev/null
!pip install selenium > /dev/null
!apt install chromium-chromedriver > /dev/null

In [2]:
from selenium import webdriver
from bs4 import BeautifulSoup
import time
from urllib.parse import quote

In [3]:
options = webdriver.ChromeOptions()
options.add_argument('--headless')   # 화면없이 실행
options.add_argument('--no-sandbox')
options.add_argument("--single-process")
options.add_argument("--disable-dev-shm-usage")
driver = webdriver.Chrome('chromedriver', options=options)

In [5]:
base_url = 'https://kin.naver.com/search/list.naver?query='
url = base_url + quote('여친선물')
driver.get(url)

In [6]:
answer_list = []
for page in range(1,11):
    print('page', page)
    url = f"{base_url}{quote('여친선물')}&page={page}"
    driver.get(url)
    time.sleep(2)

    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    lis = soup.select('ul.basic1 > li')

    for li in lis:
        sub_url = li.select_one('a._searchListTitleAnchor')['href']
        driver.get(sub_url)
        time.sleep(2)

        count = int(driver.find_element_by_css_selector('._answerCount.num').text)
        answer_page = (count - 1) // 5
        #print(count, end=', ')
        for i in range(answer_page):
            driver.find_element_by_id('nextPageButton').click()
            time.sleep(2)

        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')
        divs = soup.select('div.answer-content__item._contentWrap._answer')
        #print(len(divs))
        for div in divs:
            ps = div.select('p.se-text-paragraph')
            for p in ps:
                text = p.get_text()
                if text == ' ' or text == '\u200b':
                    continue
                answer_list.append(text)

        driver.back()
        time.sleep(2)

page 1
page 2
page 3
page 4
page 5
page 6
page 7
page 8
page 9
page 10


In [7]:
len(answer_list)

4834

In [8]:
with open('new_gift.txt', 'w', encoding='utf-8') as fp:
    for answer in answer_list:
        fp.write(answer+'\n')

In [9]:
driver.close()